# 🎬 Movie Recommendation System (Item-Based & Hybrid)

This notebook demonstrates how to build a movie recommendation system using:
1. **Item-Based Collaborative Filtering** – recommends movies similar to a given movie based on how users rated them.
2. **Content-Based Filtering** – recommends movies based on genre similarity.
3. **Hybrid Filtering** – combines both Collaborative and Content-Based methods.

### 📥 1. Load and Explore Dataset
We begin by loading the MovieLens 10M dataset which contains two main files: `movies.dat` (movie titles and genres) and `ratings.dat` (user ratings).



In [25]:
import pandas as pd

# Define column names for clarity
movie_cols = ['movieId', 'title', 'genres']
rating_cols = ['userId', 'movieId', 'rating', 'timestamp']

# Load the datasets
movies = pd.read_csv('/content/movies.dat', sep='::', names=movie_cols, engine='python')
ratings = pd.read_csv('/content/ratings.dat', sep='::', names=rating_cols, engine='python')

# checking the first 5 rows
movies.head(), ratings.head()

(   movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  ,
    userId  movieId  rating  timestamp
 0       1      122     5.0  838985046
 1       1      185     5.0  838983525
 2       1      231     5.0  838983392
 3       1      292     5.0  838983421
 4       1      316     5.0  838983392)

In [26]:
#checking the size of the datasets

movies.shape, ratings.shape

((10681, 3), (10000054, 4))

### 🧹 2. Filter Popular Movies & Active Users
We reduce memory usage by keeping only:
- Movies with at least 500 ratings
- Users who have rated at least 50 movies

In [31]:
# Filter popular movies
movie_counts = ratings['movieId'].value_counts()
popular_movies = movie_counts[movie_counts >= 500].index
ratings_filtered = ratings[ratings['movieId'].isin(popular_movies)]

# Filter active users
user_counts = ratings_filtered['userId'].value_counts()
active_users = user_counts[user_counts >= 50].index
ratings_filtered = ratings_filtered[ratings_filtered['userId'].isin(active_users)]

### 🧱 3. Build User-Movie Matrix
We create a matrix of users and movies with their corresponding ratings.

In [32]:
# Create a user-movie matrix
movie_user_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', values='rating')

# Fill missing ratings with 0 for similarity calculations
movie_user_matrix_filled = movie_user_matrix.fillna(0)

### 🔁 4. Item-Based Collaborative Filtering
We compute similarity between movies based on how users rated them.

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

# Transpose matrix to get movies as rows
movie_user_matrix_T = movie_user_matrix_filled.T

# Compute cosine similarity between movies
cosine_sim = cosine_similarity(movie_user_matrix_T)

# Build movie ID ↔ index lookup
movieId_to_index = {movieId: idx for idx, movieId in enumerate(movie_user_matrix.columns)}
index_to_movieId = {idx: movieId for movieId, idx in movieId_to_index.items()}

In [34]:
# Recommendation function using collaborative filtering
def recommend_similar_movies(movie_title, top_n=10):
    try:
        movie_id = movies[movies['title'] == movie_title]['movieId'].values[0]
    except IndexError:
        return f"Movie '{movie_title}' not found in dataset."

    if movie_id not in movieId_to_index:
        return f"Movie '{movie_title}' is not in the filtered dataset."

    idx = movieId_to_index[movie_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n + 1]

    similar_movie_ids = [index_to_movieId[i[0]] for i in sim_scores]
    similar_movies = movies[movies['movieId'].isin(similar_movie_ids)][['title', 'genres']]

    return similar_movies.reset_index(drop=True)

# Testing the function with an example input of a movie

In [37]:
# Test Collaborative Filtering
recommend_similar_movies("Toy Story (1995)", top_n=5)

,title,genres
0,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,Action|Adventure|Sci-Fi
1,Forrest Gump (1994),Comedy|Drama|Romance|War
2,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
3,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
4,Back to the Future (1985),Adventure|Comedy|Sci-Fi


In [38]:
# Test Collaborative Filtering
recommend_similar_movies("Titanic (1997)", top_n=5)

,title,genres
0,Men in Black (1997),Action|Comedy|Sci-Fi
1,Good Will Hunting (1997),Drama|Romance
2,Saving Private Ryan (1998),Action|Drama|War
3,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4,"Sixth Sense, The (1999)",Drama|Mystery|Thriller


### 🧠 5. Hybrid Recommender (Collaborative + Content-Based)
We use genres to calculate content-based similarity and then blend it with collaborative filtering.


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [40]:
# Filter movies used in CF
filtered_movie_ids = movie_user_matrix.columns
filtered_movies = movies[movies['movieId'].isin(filtered_movie_ids)].reset_index(drop=True)

In [41]:
# Re-map movie IDs to indices
movieId_to_index = {movieId: idx for idx, movieId in enumerate(filtered_movies['movieId'])}
index_to_movieId = {idx: movieId for movieId, idx in movieId_to_index.items()}

In [42]:
# Prepare genre vectors using TF-IDF
filtered_movies['genres'] = filtered_movies['genres'].fillna('')
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = tfidf.fit_transform(filtered_movies['genres'])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [43]:
# Compute genre similarity
genre_similarity = cosine_similarity(genre_matrix)

In [44]:
# Blend both similarities (collaborative + content)
alpha = 0.5
hybrid_similarity = alpha * cosine_sim + (1 - alpha) * genre_similarity

In [45]:
# Hybrid recommendation function
def hybrid_recommendations(movie_title, top_n=10):
    try:
        movie_id = filtered_movies[filtered_movies['title'] == movie_title]['movieId'].values[0]
    except IndexError:
        return f"Movie '{movie_title}' not found in dataset."

    if movie_id not in movieId_to_index:
        return f"Movie '{movie_title}' is not in the filtered dataset."

    idx = movieId_to_index[movie_id]
    sim_scores = list(enumerate(hybrid_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n + 1]

    similar_movie_ids = [index_to_movieId[i[0]] for i in sim_scores]
    similar_movies = filtered_movies[filtered_movies['movieId'].isin(similar_movie_ids)][['title', 'genres']]

    return similar_movies.reset_index(drop=True)

In [46]:
hybrid_recommendations("Titanic (1997)", top_n=5)

,title,genres
0,Casablanca (1942),Drama|Romance
1,Jerry Maguire (1996),Drama|Romance
2,Good Will Hunting (1997),Drama|Romance
3,As Good As It Gets (1997),Comedy|Drama|Romance
4,Shakespeare in Love (1998),Comedy|Drama|Romance
